# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
# Your code here

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
# Your code here

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)

# Solution pipeline 

* Choose dataset for the 
* Implement the baseline model https://norvig.com/spell-correct.html and collet metrics on test dataset.
* Test dataset: https://titan.dcs.bbk.ac.uk/~roger/holbrook-tagged.dat


# Norvig's solution

In [2]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('data/train/big.txt').read()))

In [3]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [4]:
# Read the text from the file
file_path = 'data/test/test.txt'
with open(file_path, 'r') as file:
    text = file.read()

In [5]:
text

'1.\nNIGEL THRUSH page 48 \n\n I have four in my Family Dad Mum and <ERR targ=sister> siter </ERR> .\nMy Dad works at Melton.\nMy <ERR targ=sister> siter </ERR> <ERR targ=goes> go </ERR> to Tonbury.\nMy Mum goes out <ERR targ=sometimes> some times </ERR> .\nI go to Bridgebrook i go out <ERR targ=sometimes> some times </ERR> on Tuesday night i go to Youth <ERR targ=club> clob </ERR> .\nOn thursday nights I go <ERR targ=bellringing> bell ringing </ERR> on Saturdays I go down to the farm.\non sundays I go to church.\nI go to bed at 10 o clock I <ERR targ=watch> wakh </ERR> TV at 5 o clock I live in a house.\nThe house is in the world.\nI live at Boar Parva it is near Melton and Bridgebrook and Smallerden.\nThe house is white it has stone up the <ERR targ=front> frount </ERR> it is the first from Bridgebrook and the <ERR targ=second> sexeon </ERR> from Smallerden.\nMy Mum is at home She goes to the shop on fridays My Dad goes to work at Smallerden .\n\n What I do when I get home from schoo

In [41]:
def extract_errors(text):
    # Regular expression to find error cases
    error_pattern = re.compile(r'<ERR targ=(.*?)> (.*?) </ERR>')
    errors = error_pattern.findall(text)
    return errors


def extract_context(text, n):
    # Split the text into words, excluding error tags and erroneous words
    words = re.split(r'\s+', re.sub(r'<ERR targ=.*?>.*?</ERR>', '', text))
    # Regular expression to find error cases
    error_pattern = re.compile(r'<ERR targ=(.*?)> (.*?) </ERR>')
    contexts = []

    for match in error_pattern.finditer(text):
        start, end = match.span()
        # Find the position of the error in the word list
        error_position = len(re.split(r'\s+', text[:start])) - 1
        # Extract the context before the error
        context_start = max(0, error_position - n)
        context = words[context_start:error_position]
        contexts.append((match.group(1), match.group(2), context))

    return contexts

In [20]:
# Extract errors
errors = extract_errors(text)
print("Extracted Errors:")
for error in errors:
    print(error)

Extracted Errors:
('sister', 'siter')
('sister', 'siter')
('goes', 'go')
('sometimes', 'some times')
('sometimes', 'some times')
('club', 'clob')
('bellringing', 'bell ringing')
('watch', 'wakh')
('front', 'frount')
('second', 'sexeon')
('watch', 'wach')
('watch', 'wach')
('cowboys', 'cow Boys')
('sometimes', 'some times')
('club', 'colbe')
('watch', 'wach')
('watch', 'wach')
('think', 'thing')
('TV', 'tv')
('square', 'squar')
('eyes', 'iyes')
("o'clock", 'oclock')
('knocked', 'nock')
('at', 'a')
("o'clock", 'oclock')
('killed', 'kild')
('saw', 'see')
('been', 'bean')
('knocked', 'nock')
('called', 'cald')
('came', 'cam')
('killed', 'killd')
('there', 'the')
('Her', 'Here')
('eyes', 'iyes')
('have to', 'haveto')
('before', 'be for')
('anything', 'any thing')
('else', 'als')
('wheel', 'weel')
('wheel', 'weel')
('sallies', 'sally')
('others', 'other')
('rounds', 'rouns')
('do', 'don')
('ring', 'rings')
('be', 'we')
('breaks', 'brakes')
('careful', 'carfull')
("Cynthia's", 'Cynthia')
('ga

In [42]:
# Extract contexts
n = 3  # Example context window size
contexts = extract_context(text, n)
print("\nContexts:")
for context in contexts:
    print(context)


Contexts:
('sister', 'siter', ['Dad', 'Mum', 'and'])
('sister', 'siter', ['Tonbury.', 'My', 'Mum'])
('goes', 'go', ['out', '.', 'I'])
('sometimes', 'some times', ['Tuesday', 'night', 'i'])
('sometimes', 'some times', ['Saturdays', 'I', 'go'])
('club', 'clob', ['church.', 'I', 'go'])
('bellringing', 'bell ringing', ['TV', 'at', '5'])
('watch', 'wakh', ['Smallerden.', 'The', 'house'])
('front', 'frount', ['do', 'when', 'I'])
('second', 'sexeon', ['down', 'the', 'farm'])
('watch', 'wach', ['T.V.', 'there', 'is'])
('watch', 'wach', ['.', 'The', 'Murder'])
('cowboys', 'cow Boys', ['1', 'night', 'when'])
('sometimes', 'some times', ['car.', 'The', 'body'])
('club', 'colbe', ['man', 'who', 'was'])
('watch', 'wach', ['black', 'hair', 'brown'])
('watch', 'wach', ['had', 'a', 'lot'])
('think', 'thing', ['so', 'they', 'it'])
('TV', 'tv', ['murder', 'car.', 'The'])
('square', 'squar', ['cars', 'were', 'black'])
('eyes', 'iyes', ['one', 'of', 'the'])
("o'clock", 'oclock', ['one', 'of', 'the'])
('k

# Raw code

In [ ]:
import re
import math
from collections import Counter

# === Utility Functions ===
def words(text):
    return re.findall(r'\w+', text.lower())

# === Load Corpus and Build Models ===
with open('data/big.txt', 'r') as f:
    corpus_text = f.read()

# Unigram model
WORDS = Counter(words(corpus_text))

# Build a bigram model from the corpus.
def build_bigram_model(text):
    tokens = words(text)
    bigrams = zip(tokens, tokens[1:])
    return Counter(bigrams)

BIGRAMS = build_bigram_model(corpus_text)
VOCAB_SIZE = len(WORDS)

def unigram_probability(word, N=sum(WORDS.values())):
    return WORDS[word] / N

def bigram_probability(prev, word):
    # Using add-one smoothing.
    return (BIGRAMS.get((prev, word), 0) + 1) / (WORDS[prev] + VOCAB_SIZE)

# === Norvig's Baseline Correction Functions (Unigram-based) ===
def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words_list):
    "The subset of `words_list` that appear in the dictionary of WORDS."
    return set(w for w in words_list if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def correction(word):
    "Most probable spelling correction for word using unigram frequencies."
    return max(candidates(word), key=P)

def ranked_candidates(word):
    "Return candidates sorted by probability in descending order."
    return sorted(candidates(word), key=P, reverse=True)

# === Context-Sensitive Correction (N-gram based) ===
def get_sentence_context(sentence, error_token):
    """
    Given a sentence and an erroneous token, extract the immediate left and right context.
    This function first removes error markers from the sentence.
    Returns (left_context, right_context). If not found, returns (None, None).
    """
    # Remove error markers from the sentence.
    clean_sentence = re.sub(r'<ERR\s+targ=[^>]+>', '', sentence)
    clean_sentence = re.sub(r'</ERR>', '', clean_sentence)
    tokens = words(clean_sentence)
    error_token = error_token.lower()
    if error_token in tokens:
        index = tokens.index(error_token)
        left = tokens[index - 1] if index > 0 else None
        right = tokens[index + 1] if index < len(tokens) - 1 else None
        return left, right
    return None, None

def score_candidate_ngram(candidate, left, right):
    """
    Compute a score for a candidate correction using the unigram probability
    and, if available, the bigram probabilities with left and right context.
    """
    score = unigram_probability(candidate)
    if left:
        score *= bigram_probability(left, candidate)
    if right:
        score *= bigram_probability(candidate, right)
    return score

def ranked_candidates_ngram(erroneous, sentence):
    """
    Return candidates sorted by a context-sensitive score.
    """
    left, right = get_sentence_context(sentence, erroneous)
    cand_list = list(candidates(erroneous))
    ranked = sorted(cand_list, key=lambda w: score_candidate_ngram(w, left, right), reverse=True)
    return ranked

def correction_ngram(erroneous, sentence):
    "Context-sensitive correction using n-gram probabilities."
    ranked = ranked_candidates_ngram(erroneous, sentence)
    return ranked[0] if ranked else erroneous

# === Error Instance Extraction Functions ===
def extract_error_instances(text):
    """
    Extract error instances from the text.
    Each instance is a tuple: (erroneous token, gold (target) token)
    """
    pattern = r'<ERR\s+targ=([^>]+)>\s*([^<]+)\s*</ERR>'
    matches = re.findall(pattern, text)
    return [(err.strip(), targ.strip()) for targ, err in matches]

def extract_error_instances_with_context(text):
    """
    Extract error instances along with their sentence context.
    Returns a list of tuples: (erroneous token, gold token, sentence)
    """
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    instances = []
    pattern = r'<ERR\s+targ=([^>]+)>\s*([^<]+)\s*</ERR>'
    for sentence in sentences:
        matches = re.findall(pattern, sentence)
        for targ, err in matches:
            instances.append((err.strip(), targ.strip(), sentence))
    return instances

# === Benchmark Evaluation Code ===
def evaluate_corrections(text, correction_func, use_context=False):
    """
    Evaluate the correction function on all extracted error tokens from the given text.
    Computes token-level accuracy and Mean Reciprocal Rank (MRR) if ranked candidate lists are available.
    """
    if use_context:
        error_instances = extract_error_instances_with_context(text)
    else:
        base_instances = extract_error_instances(text)
        error_instances = [(err, targ, None) for err, targ in base_instances]
    
    correct_count = 0
    total = len(error_instances)
    mrr_total = 0.0
    
    print("=== Evaluating Corrections ===")
    for erroneous, expected, sentence in error_instances:
        # Get ranked candidates depending on whether context is used.
        if use_context and sentence is not None:
            ranked = ranked_candidates_ngram(erroneous, sentence)
            predicted = ranked[0] if ranked else erroneous
        else:
            ranked = ranked_candidates(erroneous)
            predicted = ranked[0] if ranked else erroneous
        
        # Compute reciprocal rank (1-indexed). If expected is not found, RR is 0.
        try:
            rank = ranked.index(expected) + 1
            reciprocal_rank = 1 / rank
        except ValueError:
            reciprocal_rank = 0
        
        mrr_total += reciprocal_rank
        is_correct = predicted == expected
        print(f"Sentence: {sentence}")
        print(f"Input: '{erroneous}' | Ranked Candidates: {ranked} | Predicted: '{predicted}' | Expected: '{expected}'")
        print(f"Correct: {is_correct} | Reciprocal Rank: {reciprocal_rank:.2f}\n")
        if is_correct:
            correct_count += 1
    
    accuracy = (correct_count / total) if total > 0 else 0
    mrr = (mrr_total / total) if total > 0 else 0
    print(f"Token-Level Correction Accuracy: {accuracy * 100:.2f}%")
    print(f"Mean Reciprocal Rank (MRR): {mrr:.2f}")
    return accuracy, mrr

# === Sentence-Level Perplexity Evaluation ===
def sentence_perplexity(sentence):
    """
    Compute the perplexity of a sentence using the bigram model with add-one smoothing.
    """
    tokens = words(sentence)
    if len(tokens) < 2:
        return float('inf')
    log_prob = 0.0
    for i in range(1, len(tokens)):
        prev, curr = tokens[i - 1], tokens[i]
        prob = bigram_probability(prev, curr)
        log_prob += math.log(prob)
    avg_log_prob = log_prob / (len(tokens) - 1)
    perplexity = math.exp(-avg_log_prob)
    return perplexity

def evaluate_sentence_level(text, correction_func, use_context=False):
    """
    For each sentence, replace erroneous tokens with corrections and compute
    the sentence-level perplexity before and after correction.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    total_before = 0.0
    total_after = 0.0
    count = 0
    print("=== Sentence-Level Perplexity Evaluation ===")
    for sentence in sentences:
        original_sentence = sentence
        corrected_sentence = sentence
        # Find all error markers in the sentence.
        pattern = r'<ERR\s+targ=([^>]+)>\s*([^<]+)\s*</ERR>'
        matches = re.findall(pattern, sentence)
        for targ, err in matches:
            if use_context:
                correction_token = correction_ngram(err.strip(), sentence)
            else:
                correction_token = correction(err.strip())
            # Replace the error marker with the correction.
            corrected_sentence = corrected_sentence.replace(f"<ERR targ={targ}> {err} </ERR>", correction_token)
        # For the "before" version, simply remove the error markers.
        cleaned_original = re.sub(r'<ERR\s+targ=[^>]+>\s*([^<]+)\s*</ERR>', r'\1', original_sentence)
        pp_before = sentence_perplexity(cleaned_original)
        pp_after = sentence_perplexity(corrected_sentence)
        print(f"Original: {cleaned_original}")
        print(f"Corrected: {corrected_sentence}")
        print(f"Perplexity Before: {pp_before:.2f} | After: {pp_after:.2f}\n")
        total_before += pp_before
        total_after += pp_after
        count += 1
    avg_before = total_before / count if count > 0 else float('inf')
    avg_after = total_after / count if count > 0 else float('inf')
    print(f"Average Sentence Perplexity - Before Correction: {avg_before:.2f}")
    print(f"Average Sentence Perplexity - After Correction: {avg_after:.2f}")
    return avg_before, avg_after

# === Example Usage with Provided Test Data ===
if __name__ == "__main__":
    test_text = """
    I have four in my Family Dad Mum and <ERR targ=sister> siter </ERR> .
    My Dad works at Melton.
    My <ERR targ=sister> siter </ERR> <ERR targ=goes> go </ERR> to Tonbury.
    My Mum goes out <ERR targ=sometimes> some times </ERR> .
    I go to Bridgebrook i go out <ERR targ=sometimes> some times </ERR> on Tuesday night i go to Youth <ERR targ=club> clob </ERR> .
    On thursday nights I go <ERR targ=bellringing> bell ringing </ERR> on Saturdays I go down to the farm.
    on sundays I go to church.
    I go to bed at 10 o clock I <ERR targ=watch> wakh </ERR> TV at 5 o clock I live in a house.
    The house is in the world.
    I live at Boar Parva it is near Melton and Bridgebrook and Smallerden.
    The house is white it has stone up the <ERR targ=front> frount </ERR> it is the first from Bridgebrook and the <ERR targ=second> sexeon </ERR> from Smallerden.
    My Mum is at home She goes to the shop on fridays My Dad goes to work at Smallerden .
    
    What I do when I get home from school.
    On monday I sometimes go down the farm in the night I <ERR targ=watch> wach </ERR> TV there is BBC and I.T.V.
    I like I.T.V.
    We call Anglia I.T.V.
    We have got Anglia like to <ERR targ=watch> wach </ERR> <ERR targ=cowboys> cow Boys </ERR> .
    On Tuesday I get off the bus and <ERR targ=sometimes> some times </ERR> in the night I go to the Youth <ERR targ=club> colbe </ERR> .
    I like to <ERR targ=watch> wach </ERR> T.V.
    there is a lot of things on T.V.
    I <ERR targ=watch> wach </ERR> it each night.
    I <ERR targ=think> thing </ERR> <ERR targ=TV> tv </ERR> is good but people say it gives us <ERR targ=square> squar </ERR> <ERR targ=eyes> iyes </ERR>
    """
    
    print("=== Baseline Unigram Correction Evaluation ===")
    evaluate_corrections(test_text, correction, use_context=False)
    
    print("\n=== Context-Sensitive N-gram Correction Evaluation ===")
    evaluate_corrections(test_text, correction_ngram, use_context=True)
    
    print("\n=== Sentence-Level Perplexity Evaluation (Baseline) ===")
    evaluate_sentence_level(test_text, correction, use_context=False)
    
    print("\n=== Sentence-Level Perplexity Evaluation (Context-Sensitive) ===")
    evaluate_sentence_level(test_text, correction_ngram, use_context=True)